In [2]:
# https://colab.research.google.com/drive/1ZZXnCjFEOkp_KdNcNabd14yok0BAIuwS#forceEdit=true&sandboxMode=true&scrollTo=wdPxFvHdTLRK

import tensorflow as tf
from tensorflow import keras
from keras import datasets, layers, models, optimizers 
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import pandas as pd
from PIL import Image
from log_training import TrainingLogger

In [3]:
#  LOAD AND SPLIT DATASET
path_train = r'C:\Users\oswal\Desktop\ImageClassification\elpv-dataset\train_with_bad_images\**\*'
path_test = r'C:\Users\oswal\Desktop\ImageClassification\elpv-dataset\test_images\**\*'

# Prepare image data 
def img_to_np(path, resize = True, extract_labels=False):  
    img_array = []
    labels = []
    fpaths = glob.glob(path, recursive=True)
    for fname in fpaths:
        if(extract_labels): 
            if '_bad' in os.path.basename(fname):
                labels.append(1)  # 1 for outlier
            else:
                labels.append(0)  # 0 for non-outlier
        img = Image.open(fname).convert("L") # Grayscale when using "RGB" you have to change the encoder and decoder 
        if(resize): img = img.resize((64,64))
        img_array.append(np.asarray(img))
    images = np.array(img_array)
    if(extract_labels): return images, np.array(labels)
    return images

test_images, test_labels = img_to_np(path_test, extract_labels=True)
train_images, train_labels = img_to_np(path_train, extract_labels=True)

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

class_names = ['good', 'bad']

# Let's look at a one image
IMG_INDEX = 7  # change this to look at other images
#plt.imshow(train_images[IMG_INDEX] ,cmap=plt.cm.binary)
#plt.xlabel(class_names[train_labels[IMG_INDEX]])
#plt.show() # uncomment for prview of image


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\oswal\\Desktop\\ImageClassification\\elpv-dataset\\test_images\\bad'

In [4]:
EPOCHS=10
OPTIMIZER="adam" #change manually !!!
BATCH_SIZE=32
LEARNING_RATE=1e-3

In [4]:
# building the Convolutional Base
model = models.Sequential()
#  32, 32, 3 means we will process 32 filters of size 3x3 | imput shape 64, 64, 1 means 64x64 pixels with one chanell for RGB change to 3
model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(64, 64, 1)))
# pooling layses to downsample our feature maps and reduce their dimensions
model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid", data_format=None)) # standard pooling layer
model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid", data_format=None))
model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
# Add dense layers to classify into two classes true false 
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
# Last layer with two neurons to classify between true and false 
model.add(layers.Dense(2))

#model.summary()

In [ ]:
model.compile(optimizer=optimizers.Adam(learning_rate=LEARNING_RATE),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
# Train model with validation split
history = model.fit(train_images, train_labels, epochs=EPOCHS, batch_size=BATCH_SIZE,
                    validation_split=0.2)

#history = model.fit(train_images, train_labels, epochs=EPOCHS, batch_size=BATCH_SIZE,
#                    validation_data=(test_images, test_labels))

NameError: name 'model' is not defined

In [ ]:
# Evaluating the Model
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
# Log all the data and store it
logger = TrainingLogger(epochs=EPOCHS, learning_rate=LEARNING_RATE, batch_size=BATCH_SIZE, optimizer=OPTIMIZER)
logger.capture_model_summary(model)
logger.update_train_metrics(val_accuracy=history.history['val_accuracy'], val_loss=history.history['val_loss'], accuracy=history.history['accuracy'], loss=history.history['val_loss'])
logger.update_test_metrics(accuracy=test_acc, loss=test_loss)
logger.print_and_save_log()